In [ ]:
import argparse
import os

from pathlib import Path

import torch
from torch.backends import cudnn

from tqdm import tqdm

from utils.checkpoint import save_checkpoint
from datasets.vessel_set import VesselSet
from modules.edm import EDMLoss, EDMPrecond
import numpy as np


c:\Users\User\miniconda3\envs\pytorch\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
files = os.listdir("centerlineArray")

In [9]:
all_vessels = []

for file in files:
    if file.endswith(".npy"):
        aneu = np.load("centerlineArray/" + file)
        segment_numbers = np.unique(aneu[:, -1])

        # Collect segments for this vessel
        segments = []
        for seg_num in segment_numbers:
            # Filter rows for the current segment and remove the segment number column
            segment_points = aneu[aneu[:, -1] == seg_num][:, :-1]
            segments.append(segment_points)
        
        # Find the maximum number of points in any segment for this vessel
        max_points = max(segment.shape[0] for segment in segments)

        # Pad each segment to have the same number of points
        padded_segments = [
            np.pad(segment, ((0, max_points - segment.shape[0]), (0, 0)), 'constant')
            for segment in segments
        ]

        # Stack all segments for this vessel along a new dimension (axis 0)
        vessel_data = np.stack([padded_segments], axis=0)
        
        # Add the stacked segments for this vessel to the list of all vessels
        all_vessels.append(vessel_data)

max_segments = max(arr.shape[1] for arr in all_vessels)
max_points = max(arr.shape[2] for arr in all_vessels)


print(all_vessels[0].shape)
print(all_vessels[1].shape)
# Pad each array to have the shape (1, max_segments, max_points, 4)
padded_arrays = []
for arr in all_vessels:
    # Calculate padding for the segments and points dimensions
    pad_segments = max_segments - arr.shape[1]
    pad_points = max_points - arr.shape[2]
    
    # Pad the array
    padded_arr = np.pad(
        arr,
        pad_width=((0, 0), (0, pad_segments), (0, pad_points), (0, 0)),
        mode='constant',
        constant_values=0  # You can change this if needed
    )
    padded_arrays.append(padded_arr)

# Stack the padded arrays along a new dimension
final_data = np.concatenate(padded_arrays, axis=0)

print("Final data shape:", final_data.shape)  # Expected shape: (number_of_arrays, max_segments, max_points, 4)
np.save("datasets/aneurisk/data_train.npy", final_data)

(1, 7, 1215, 4)
(1, 5, 1041, 4)
Final data shape: (98, 12, 1615, 4)


In [13]:
train_set = VesselSet(split="train", path="aneurisk")

train_loader = torch.utils.data.DataLoader(
    dataset=train_set,
    batch_size=16,
    num_workers=0,
    drop_last=True,
    shuffle=True,
)

In [14]:
for inputs, labels in train_loader:
    print(inputs.shape)

torch.Size([16, 256, 8])
torch.Size([16, 256, 8])
torch.Size([16, 256, 8])
torch.Size([16, 256, 8])
torch.Size([16, 256, 8])
torch.Size([16, 256, 8])


In [16]:
import matplotlib.pyplot as plt

In [20]:
for i, batch in enumerate(train_loader):
    print(len(batch))
    for tree in batch:
        print(tree.shape)
        segment = np.concatenate(tree, axis=0)
            

        segment[..., :3] -= segment[..., :3].mean(0, keepdims=True)
        print(segment)
        max_r = np.max(np.abs(segment))

        points, radii = segment[..., :3], segment[..., 3]

        fig = plt.figure(figsize=(10, 8))

        ax = fig.add_subplot(projection="3d")
        ax.scatter(0, 0, 0, color="red", s=50)
        ax.scatter(*points[...,].T, s=(10 * radii) ** 2, c=points[..., 2])

        ax.set_xlim(-max_r, max_r)
        ax.set_ylim(-max_r, max_r)
        ax.set_zlim(-max_r, max_r)

        plt.show()

2
torch.Size([16, 256, 8])


TypeError: The first input argument needs to be a sequence